In [ ]:
from requests import get
import json
def get_user_friends(user_id):
    response = get(f'https://api.vk.com/method/friends.get?v=5.131&access_token={token}&user_id={user_id}&fields=nickname')
    try:
        toRet = [{'id': pers['id'], 'name': f"{pers['first_name']} {pers['last_name']}"} for pers in json.loads(response.text)['response']['items']]
    except KeyError:
        toRet = []
    return toRet

In [ ]:
import vk_api
import networkx as nx
token='3de6e36b3de6e36b3de6e36b293ef433c533de63de6e36b5e1e1cba36999d0d15f65ef8'
# Создаем объект vk_session и получаем объект VK API
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

# Задаем список пользователей, для которых мы будем строить граф
user_ids = [17958998,   # Апальков Никита
    430899909,  # Асямолов Антон
    167840516,  # Верников Сергей
    474712276,  # Видякин Александр
    132187153,  # Власков Александр
    93734430,   # Кантемиров Роман
    169871363,  # Кириллов Григорий
    211222538,  # Кудряшев Роман
    78087447,   # Назаров Тимур
    346137565,  # Ознобихин Виталий
    54575378,   # Покровская Анна
    220270939,  # Сорокин Николай
    217432918,  # Толокнова Владлена
    155827251,  # Ципигин Дмитрий
    203613697 ,  # Яшин Никита
    ]

In [ ]:
# Создаем пустой граф
G = nx.Graph()

G.add_nodes_from([
    (17958998, {"name": "Апальков Никита"}),
    (430899909, {"name": "Асямолов Антон"}),
    (167840516, {"name": "Верников Сергей"}),
    (474712276, {"name": "Видякин Александр"}),
    (132187153, {"name": "Власков Александр"}),
    (93734430, {"name": "Кантемиров Роман"}),
    (169871363, {"name": "Кириллов Григорий"}),
    (211222538, {"name": "Кудряшев Роман"}),
    (78087447, {"name": "Назаров Тимур"}),
    (346137565, {"name": "Ознобихин Виталий"}),
    (54575378, {"name": "Покровская Анна"}),
    (220270939, {"name": "Сорокин Николай"}),
    (217432918, {"name": "Толокнова Владлена"}),
    (155827251, {"name": "Ципигин Дмитрий"}),
    (203613697, {"name": "Яшин Никита"}),
])
#  Для каждого пользователя из списка user_ids
for user_id in user_ids:
    try:
        #  Получаем список друзей пользователя
        friends = get_user_friends(user_id)

        #  Для каждого друга пользователя
        for friend in friends:
            try:
                #  Получаем список друзей друга пользователя
                friends2 = get_user_friends(friend['id'])

                #  Добавляем друга пользователя в граф
                G.add_nodes_from([(friend['id'], {"name": friend['name']})])

                #  Добавляем ребро между пользователем и его другом
                G.add_edge(user_id, friend['id'])

                #  Для каждого друга друга пользователя
                for friend_friend in friends2:
                    try:
                        #  Добавляем друга друга пользователя в граф
                        G.add_nodes_from([(friend_friend['id'], {"name": friend_friend['name']})])

                        #  Добавляем ребро между другом пользователя и его другом друга
                        G.add_edge(friend['id'], friend_friend['id'])
                    except:
                        pass
            except:
                pass
    except:
        pass

In [ ]:
# Сохраняем граф в файл при помощи модуля pickle
import pickle
with open('graph.pickle', 'wb') as f:
    pickle.dump(G, f)

In [ ]:
pos = nx.spring_layout(G)
nx.draw(G, pos, node_color='lightblue', with_labels=True, font_size=10, node_size=500, alpha=0.8)

In [ ]:
#Центральность посредничество
between_cent = nx.betweenness_centrality(G)
print(between_cent)

In [ ]:
#Центральность степенная
degree_centrality = nx.degree_centrality(G)
max(degree_centrality.values())

In [ ]:
#Центральность близость
close_cent = nx.closeness_centrality(G)
print(close_cent)

In [ ]:
#Центральность собственный вектор
eigen_cent = nx.eigenvector_centrality(G)
print(eigen_cent)

In [ ]:
#Диаметр
d = nx.diameter(G)
print(d)

In [ ]:
#Радиус
r = nx.radius(G)
print(r)

In [ ]:
#Центральные вершины
centr = nx.center(G)
print(centr)

In [ ]:
#Периферийные вершины
perif = nx.periphery(G)
print(perif)

In [ ]:
#Максимум клики
max_cliques = nx.find_cliques(G)
print(max_cliques)

In [ ]:
#Эксцентриситет
ecc = nx.eccentricity(G)

In [ ]:
# проверяем граф на триангуляцию
is_chordal = nx.is_chordal(G)
if is_chordal:
    print("Граф триангулирован")
else:
    print("Граф не триангулирован")

In [ ]:
# приводим граф к триангулированному виду
triangulated_graph = nx.chordal_graph_cliques(G)